In [14]:
import matplotlib.pyplot as plt
import torch

from torch import nn
from torch import optim

from torch.nn import functional as nn_f
from torch.utils.data import DataLoader

from torchvision import transforms, models
from torchvision.datasets import ImageFolder

In [21]:
this_device = torch.device("cuda" if torch.cuda.is_available else "cpu")

In [22]:
transformations = transforms.Compose([
    transforms.Resize((224, 244)),
    transforms.ToTensor(),
    transforms.Normalize(
        mean=[0.485, 0.456, 0.406],
        std=[0.229, 0.224, 0.225]
    )
])

In [23]:
train_data = ImageFolder("../../data/catsndogs/training_set", transform=transformations)
test_data = ImageFolder("../../data/catsndogs/test_set", transform=transformations)

In [24]:
class CDNNet(nn.Module):
    
    def __init__(self):
        super().__init__()
        
        self.conv1 = nn.Conv2d(3, 32, kernel_size=3, padding=1)
        self.pool1 = nn.MaxPool2d(2)
         
        self.conv2 = nn.Conv2d(32, 64, kernel_size=3, padding=1)
        self.pool2 = nn.MaxPool2d(2)
        
        self.conv3 = nn.Conv2d(64, 128, kernel_size=3, padding=1)
        self.pool3 = nn.MaxPool2d(2)
        
        self.flatten = nn.Flatten()
        self.fc1 = nn.Linear(128*28*28, 256)
        self.fc2 = nn.Linear(256, 128)
        self.fc3 = nn.Linear(128, 1)
        
    def forward(self, x):
        out = self.pool1(nn_f.relu(self.conv1(x)))
        out = self.pool2(nn_f.relu(self.conv2(out)))
        out = self.pool3(nn_f.relu(self.conv3(out)))
        
        out = self.flatten(out)
        
        out = nn_f.relu(self.fc1(out))
        out = nn_f.relu(self.fc2(out))
        out = nn_f.sigmoid(self.fc3(out))
        
        return out

In [25]:
def training_loop(n_epochs, optimizer, model, loss_fn, train_loader, test_loader, print_plot):
    train_accuracy = torch.zeros(n_epochs)
    test_accuracy = torch.zeros(n_epochs)
    
    for epoch in range(n_epochs):
        for example, label in train_loader:
            example = example.to(this_device)
            label = label.to(this_device).unsqueeze(1)
            output = model(example.float())
            
            loss = loss_fn(output, label.float())
            
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            
        with torch.no_grad():
            for loader, accuracy in [(train_loader, train_accuracy), (test_loader, test_accuracy)]:
                correct = 0
                total = 0
                
                for examples, labels in loader:
                    examples = examples.to(device=this_device)
                    labels = labels.to(device=this_device).view(-1, 1)
                    
                    output = model(examples.float())
                    prediction = torch.round(output)
                    
                    total += labels.shape[0]
                    correct += (prediction == labels).sum()
                    
                accuracy[epoch] = correct/total
                
        
        print(f"Epoch {epoch + 1}\n",
              f"Train Accuracy: {train_accuracy[epoch]}\n",
              f"Test Accuracy: {test_accuracy[epoch]}")
    
    if print_plot:
        epochs = range(n_epochs)
        
        plt.plot(epochs, train_accuracy, 'g', label="Training accuracy")
        plt.plot(epochs, test_accuracy, 'b', label="Test accuracy")
        plt.title("Training and Test accuracy")
        plt.xlabel("Epochs")
        plt.ylabel("Accuracy")
        plt.legend()
        plt.show()

In [26]:
train_loader = DataLoader(train_data, batch_size=32, shuffle=True)
test_loader = DataLoader(test_data, batch_size=32, shuffle=True)
learning_rate = 1e-3

In [27]:
model = CDNNet().to(this_device)
optimizer = optim.Adam(model.parameters(), lr=learning_rate)
loss_fn = nn.BCELoss()

training_loop(
    n_epochs=10,
    optimizer=optimizer,
    model=model,
    loss_fn=loss_fn,
    print_plot=True,
    train_loader=train_loader,
    test_loader=test_loader
)

AssertionError: Torch not compiled with CUDA enabled